In [3]:
import numpy as np
from scipy import fft as fft
from scipy.sparse.linalg import LinearOperator
from matplotlib import pyplot as plt
  
plt.rcParams['figure.figsize'] = [8, 4]
plt.rcParams['figure.dpi'] = 200 # 200 e.g. is really fine, but slower

class FourierMultiplier(LinearOperator):

    def __init__(self, N, L=1, transform='dct'):
        self.L = L
        self.N = N
        self.shape = (self.N,self.N)
        self.dtype = None
        self.freqs = fft.fftfreq(self.N, d=self.L/self.N)
        self.x = np.linspace(0, self.L, num=self.N, endpoint=False)
        self.multiplier = None ## Need to implement this on particular case
        self.transform = transform
        if self.transform == 'dct':
            self.fwd = fft.dct
            self.inv = fft.idct
            self.func = np.cos
        elif self.transform == 'dst':
            self.fwd = fft.dst
            self.inv = fft.idst
            self.func = np.sin
        elif self.transform == 'fft':
            self.fwd = np.fft.fft
            self.inv = np.fft.ifft
            self.func = lambda x: np.exp(2j*x)
        
    def to_freq_domain(self, u):
        return self.fwd(u)/self.N
    
    def to_time_domain(self, u):
        return self.inv(u) * self.N
    
    def coeff2u(self, coeff):
        res = sum(coeff[i]*self.func(np.pi*self.freqs[i]*self.x) for i in np.where(coeff)[0])
        return res    
    
    def _matvec(self, v):
        return self(np.squeeze(v))
    
    def __call__(self, v):
        v_hat = self.to_freq_domain(v)
        Av_hat = v_hat * self.multiplier
        Av = self.to_time_domain(Av_hat) 
        return Av + np.mean(v)

    def random(self, initial_coeffs=False, beta=1.137):
        if self.transform == 'fft':
            coeffs = np.random.randn(self.N, 2).view(np.complex128) 
            coeffs = np.squeeze(coeffs)
        else:
            coeffs = np.random.randn(self.freqs.size)
        coeffs /= (np.arange(coeffs.size)+1)**beta
        coeffs[0] = 0
        u0 = self.coeff2u(coeffs) 
        return u0, coeffs if initial_coeffs else u0 

    
    
class Heat(FourierMultiplier):
    def __init__(self, time, alpha=1, **kwargs):
        super().__init__(**kwargs)
        self.alpha = alpha
        self.time = time
        self.multiplier = np.exp(-self.alpha * np.pi**2 * self.time * self.freqs**2)
          

class Laplacian(FourierMultiplier):
    def __init__(self, alpha, **kwargs):
        super().__init__(**kwargs)
        self.multiplier = np.power(np.pi**2 * self.freqs**2, alpha)
        self.multiplier[0] = 1
      

    def __call__(self, v):
        return super().__call__(v)# - v[0]
        
   

Using matplotlib backend: GTK3Agg


In [ ]:
N = 100
L = 2 * np.pi
time = 2e-3
alpha = 2.6
transform = 'dct'

fwd = Heat(N=N, L=L, alpha=alpha, time=time, transform=transform)
u0, coeffs = fwd.random(initial_coeffs=True, beta=1.4)

# Analytic
coeffs = coeffs * fwd.multiplier
uT = fwd.coeff2u(coeffs)

# Numeric solution
uT_num = fwd(u0)
        
lap = Laplacian(alpha=alpha, N=N, L=L, transform=transform)
invlap = Laplacian(alpha=-alpha, N=N, L=L, transform=transform)

plt.plot(fwd.x, u0.real, label='IC')
plt.plot(fwd.x, uT.real+0.005, label='Analytic FC')
plt.plot(fwd.x, uT_num.real-0.005, label='Numeric FC')
plt.plot(lap.x, lap(invlap(uT))+0.015, label=r'$\Delta\Delta^{-1} FC$')
plt.plot(lap.x, invlap(lap(uT))-0.015, label=r'$\Delta^{-1}\Delta FC$')
plt.legend()
#plt.close()